# Сравнение хранилищ ClickHouse и Vertica

In [1]:
import uuid
import random 
import timeit
import time
from clickhouse_driver import Client
import vertica_python

### Подготовка данных для теста

In [2]:
file = open('insert_file.csv','w')

In [3]:
n = 1
while n <= 10000000:
    file.write(f'{n},{uuid.uuid1()},{uuid.uuid1()},{random.randint(1,20000)}\n')
    n+=1

# Clickhouse

In [4]:
client = Client(host='127.0.0.1') 

### Создание базы test_base

In [5]:
client.execute('CREATE DATABASE IF NOT EXISTS test_base ON CLUSTER company_cluster')

[('clickhouse-node5', 9000, 0, '', 5, 0),
 ('clickhouse-node1', 9000, 0, '', 4, 0),
 ('clickhouse-node3', 9000, 0, '', 3, 0),
 ('clickhouse-node4', 9000, 0, '', 2, 0),
 ('clickhouse-node6', 9000, 0, '', 1, 0),
 ('clickhouse-node2', 9000, 0, '', 0, 0)]

### Создание таблицы test_table

In [6]:
client.execute('CREATE TABLE IF NOT EXISTS test_base.test_table ON CLUSTER company_cluster (id Int64, movie_id UUID, user_id UUID, frame Int64) Engine=MergeTree() ORDER BY id')

[('clickhouse-node5', 9000, 0, '', 5, 0),
 ('clickhouse-node1', 9000, 0, '', 4, 0),
 ('clickhouse-node3', 9000, 0, '', 3, 0),
 ('clickhouse-node6', 9000, 0, '', 2, 0),
 ('clickhouse-node4', 9000, 0, '', 1, 0),
 ('clickhouse-node2', 9000, 0, '', 0, 0)]

### Вставка из файла

In [7]:
file = open('insert_file.csv','r').read()

In [8]:
start_time = time.time()
client.execute(f"INSERT INTO test_base.test_table (id, movie_id, user_id, frame) FORMAT CSV {file}")
clickhouse_insert = f"Clickhouse вставка из файла: {(time.time() - start_time)}"
print(clickhouse_insert)

Clickhouse вставка из файла: 40.03273797035217


### Вставка нескольких строк

In [9]:
start_time = time.time()
client.execute(f'''INSERT INTO test_base.test_table (id, movie_id, user_id, frame) VALUES
               (1,'e67bfbe0-0b3f-11ed-98f2-acde48001122','e67bfe2e-0b3f-11ed-98f2-acde48001122',339382096),
               (2,'e67bff14-0b3f-11ed-98f2-acde48001122','e67bff6e-0b3f-11ed-98f2-acde48001122',151403798),
               (3,'e67bffd2-0b3f-11ed-98f2-acde48001122','e67c0018-0b3f-11ed-98f2-acde48001122',370468578)''')
clickhouse_insert3 = f"Clickhouse вставка 3 строк: {(time.time() - start_time)}"
print(clickhouse_insert3)

Clickhouse вставка 3 строк: 0.009743928909301758


### Многократная вставка нескольких строк с использованием timeit

In [10]:
# код для timeit
timeit_code = """client.execute("INSERT INTO test_base.test_table (id, movie_id, user_id, frame) VALUES (1,'e67bfbe0-0b3f-11ed-98f2-acde48001122','e67bfe2e-0b3f-11ed-98f2-acde48001122',339382096), (2,'e67bff14-0b3f-11ed-98f2-acde48001122','e67bff6e-0b3f-11ed-98f2-acde48001122',151403798), (3,'e67bffd2-0b3f-11ed-98f2-acde48001122','e67c0018-0b3f-11ed-98f2-acde48001122',370468578)")"""

In [11]:
elapsed_time = timeit.timeit(timeit_code, number=1000,globals=globals())/1000

clickhouse_insert3_iterate = f"Clickhouse cреднее время вставки 3 строк 1000 раз: {elapsed_time})"
print(clickhouse_insert3_iterate)

Clickhouse cреднее время вставки 3 строк 1000 раз: 0.01030745153799998)


### Вставка 500 строк

In [12]:
insert_list = [[i,uuid.uuid1(),uuid.uuid1(),random.randint(1,20000)] for i in range(500)]

In [13]:
start_time = time.time()
client.execute(f"INSERT INTO test_base.test_table (id, movie_id, user_id, frame) VALUES", 
                (tuple(row) for row in insert_list))
clickhouse_insert500 = f"Clickhouse время вставки 500 строк: {(time.time() - start_time)}"
print(clickhouse_insert500)

Clickhouse время вставки 500 строк: 0.012792825698852539


### Многократная вставка 500 строк с использованием timeit

In [14]:
timeit_code = """client.execute(f"INSERT INTO test_base.test_table (id, movie_id, user_id, frame) VALUES", 
                (tuple(row) for row in insert_list))"""

In [15]:
elapsed_time = timeit.timeit(timeit_code, number=1000,globals=globals())/1000
clickhouse_insert500_iterate = f"Clickhouse cреднее время вставки 500 строк 1000 раз: {elapsed_time}"
print(clickhouse_insert500_iterate)

Clickhouse cреднее время вставки 500 строк 1000 раз: 0.011356499548000016


## Запросы

### Запрос всех данных

In [16]:
start_time = time.time()
result = client.execute('SELECT * FROM test_base.test_table')
res_time = time.time() - start_time
print(f"Clickhouse время запроса всех строк: {res_time}")

Clickhouse время запроса всех строк: 60.1036102771759


In [17]:
clickhouse_select_all = f"Clickhouse время запроса всех строк: {res_time} kоличество строк в ответе: {len(result)}"
print(clickhouse_select_all)
result.clear()

Clickhouse время запроса всех строк: 60.1036102771759 kоличество строк в ответе: 10503428


### Запрос по movie_id

In [18]:
start_time = time.time()
result = client.execute("SELECT * FROM test_base.test_table WHERE movie_id =='e67bfbe0-0b3f-11ed-98f2-acde48001122'")
res_time = time.time() - start_time
clickhouse_select_movie_id = f"Clickhouse время запроса по конкретному movie_id: {res_time} количество строк в ответе: {len(result)}"

print(clickhouse_select_movie_id)
result.clear()

Clickhouse время запроса по конкретному movie_id: 0.08955597877502441 количество строк в ответе: 1001


### Запрос с фильтацией по frame

In [19]:
start_time = time.time()
result = client.execute("SELECT * FROM test_base.test_table WHERE frame > 10000")
res_time = time.time() - start_time
clickhouse_select_frame1 = f"Clickhouse время запроса frame > 1000: {res_time} количество строк в ответе: {len(result)}"

print(clickhouse_select_frame1)
result.clear()

Clickhouse время запроса frame > 1000: 27.881654024124146 количество строк в ответе: 5253700


### Запрос с фильтацией по frame с меньшим количеством строк в ответе

In [20]:
start_time = time.time()
result = client.execute("SELECT * FROM test_base.test_table WHERE frame > 100 and frame < 300")
res_time = time.time() - start_time
clickhouse_select_frame100_300 = f"Clickhouse время запроса frame > 100 and < 300: {res_time} количество строк в ответе: {len(result)}"

print(clickhouse_select_frame100_300)
result.clear()

Clickhouse время запроса frame > 100 and < 300: 0.6122901439666748 количество строк в ответе: 103581


### Предыдущий запрос c order by

In [21]:
start_time = time.time()
result = client.execute("SELECT * FROM test_base.test_table WHERE frame > 100 and frame <300 order by frame")
res_time = time.time() - start_time
clickhouse_select_frame100_300_order = f"Clickhouse время запроса frame > 100 and < 300 order: {res_time} количество строк в ответе: {len(result)}"

print(clickhouse_select_frame100_300_order)
result.clear()

Clickhouse время запроса frame > 100 and < 300 order: 0.748755931854248 количество строк в ответе: 103581


### Удаление таблицы test_base.test_table

In [22]:
client.execute('DROP TABLE IF EXISTS test_base.test_table ON CLUSTER company_cluster')

[('clickhouse-node5', 9000, 0, '', 5, 1),
 ('clickhouse-node3', 9000, 0, '', 4, 1),
 ('clickhouse-node2', 9000, 0, '', 3, 1),
 ('clickhouse-node4', 9000, 0, '', 2, 1),
 ('clickhouse-node6', 9000, 0, '', 1, 1),
 ('clickhouse-node1', 9000, 0, '', 0, 0)]

# Vertica

In [23]:
connection_info = {
    'host': '127.0.0.1',
    'port': 5433,
    'user': 'dbadmin',
    'password': '',
    'database': 'docker',
    'autocommit': True,
}

In [24]:
connection = vertica_python.connect(**connection_info)
cursor = connection.cursor()

In [25]:
cursor.execute("""CREATE TABLE test (
        id IDENTITY,
        id_test INTEGER,
        movie_id UUID,
        user_id UUID,
        frame INTEGER NOT NULL);
    """)

### Вставка из файла

In [26]:
start_time = time.time()
cursor.execute("COPY test FROM LOCAL 'insert_file.csv' PARSER fcsvparser();")
vertica_insert = f"Vertica вставка из файла: {(time.time() - start_time)}"
print(vertica_insert)

Vertica вставка из файла: 38.8112678527832


### Вставка нескольких строк

In [27]:
start_time = time.time()
cursor.executemany('INSERT INTO test (id_test, movie_id, user_id, frame) VALUES (?,?,?,?)', 
                  [(1,'e67bfbe0-0b3f-11ed-98f2-acde48001122','e67bfe2e-0b3f-11ed-98f2-acde48001122',339382096),
                   (2,'e67bff14-0b3f-11ed-98f2-acde48001122','e67bff6e-0b3f-11ed-98f2-acde48001122',151403798),
                   (3,'e67bffd2-0b3f-11ed-98f2-acde48001122','e67c0018-0b3f-11ed-98f2-acde48001122',370468578)],
                    use_prepared_statements=True)
vertica_insert3 = f"Vertica вставка 3 строк: {(time.time() - start_time)}"
print(vertica_insert3)

Vertica вставка 3 строк: 0.14068317413330078


### Многократная вставка нескольких строк с использованием timeit

In [28]:
# код для timeit
timeit_code = """cursor.executemany('INSERT INTO test (id_test, movie_id, user_id, frame) VALUES (?,?,?,?)', 
                  [(1,'e67bfbe0-0b3f-11ed-98f2-acde48001122','e67bfe2e-0b3f-11ed-98f2-acde48001122',339382096),
                   (2,'e67bff14-0b3f-11ed-98f2-acde48001122','e67bff6e-0b3f-11ed-98f2-acde48001122',151403798),
                   (3,'e67bffd2-0b3f-11ed-98f2-acde48001122','e67c0018-0b3f-11ed-98f2-acde48001122',370468578)],
                    use_prepared_statements=True)"""

In [29]:
elapsed_time = timeit.timeit(timeit_code, number=1000,globals=globals())/1000

vertica_insert3_iterate = f"Vertica cреднее время вставки 3 строк 1000 раз: {elapsed_time})"
print(vertica_insert3_iterate)

Vertica cреднее время вставки 3 строк 1000 раз: 0.043370120616000006)


### Вставка 500 строк

In [30]:
insert_list = [(i,uuid.uuid1(),uuid.uuid1(),random.randint(1,20000)) for i in range(500)]

In [31]:
start_time = time.time()
cursor.executemany('INSERT INTO test (id_test, movie_id, user_id, frame) VALUES (?,?,?,?)', 
                insert_list,
                use_prepared_statements=True)

vertica_insert500 = f"Vertica время вставки 500 строк: {(time.time() - start_time)}"
print(vertica_insert500)

Vertica время вставки 500 строк: 4.9783289432525635


### Многократная вставка 500 строк с использованием timeit

In [32]:
# код для timeit
timeit_code = """cursor.executemany('INSERT INTO test (id_test, movie_id, user_id, frame) VALUES (?,?,?,?)', 
                insert_list,
                use_prepared_statements=True)"""

In [33]:
elapsed_time = timeit.timeit(timeit_code, number=10,globals=globals())/10

vertica_insert500_iterate = f"Vertica cреднее время вставки 500 строк 10 раз: {elapsed_time})"
print(vertica_insert500_iterate)
# время выполнение блока при параметре number=1000 примерно 4000 секунд а это больше часа... 

Vertica cреднее время вставки 500 строк 10 раз: 6.752269320799996)


## Запросы

### Запрос всех данных

In [34]:
start_time = time.time()
result = cursor.execute('SELECT * FROM test').fetchall()
res_time = time.time() - start_time
print(f"Vertica время запроса всех строк: {res_time}")

Vertica время запроса всех строк: 244.79380202293396


In [35]:
vertica_select_all = f"Vertica время запроса всех строк: {res_time} kоличество строк в ответе: {len(result)}"
print(vertica_select_all)
result.clear()

Vertica время запроса всех строк: 244.79380202293396 kоличество строк в ответе: 10008502


### Запрос по movie_id

In [36]:
start_time = time.time()
query = cursor.execute("SELECT * FROM test WHERE movie_id ='e67bfbe0-0b3f-11ed-98f2-acde48001122'")
result= query.fetchall()
res_time = time.time() - start_time
vertica_select_movie_id = f"Vertica время запроса по конкретному movie_id: {res_time} количество строк в ответе: {len(result)}"

print(vertica_select_movie_id)
result.clear()

Vertica время запроса по конкретному movie_id: 0.08078789710998535 количество строк в ответе: 1001


### Запрос с фильтацией по frame

In [37]:
start_time = time.time()
query = cursor.execute("SELECT * FROM test WHERE frame > 10000")
result= query.fetchall()
res_time = time.time() - start_time
vertica_select_frame1 = f"Vertica время запроса по конкретному movie_id: {res_time} количество строк в ответе: {len(result)}"

print(vertica_select_frame1)
result.clear()

Vertica время запроса по конкретному movie_id: 131.91403198242188 количество строк в ответе: 5006330


### Запрос с фильтацией по frame с меньшим количеством строк в ответе

In [38]:
start_time = time.time()
query = cursor.execute("SELECT * FROM test WHERE frame > 100 and frame < 300")
result= query.fetchall()
res_time = time.time() - start_time
vertica_select_frame100_300 = f"Vertica время запроса frame > 100 and < 300: {res_time} количество строк в ответе: {len(result)}"

print(vertica_select_frame100_300)
result.clear()

Vertica время запроса frame > 100 and < 300: 2.38722825050354 количество строк в ответе: 99676


### Предыдущий запрос c order by

In [39]:
start_time = time.time()
query = cursor.execute("SELECT * FROM test WHERE frame > 100 and frame < 300 order by frame")
result= query.fetchall()
res_time = time.time() - start_time
vertica_select_frame100_300_order = f"Vertica время запроса frame > 100 and < 300 order: {res_time} количество строк в ответе: {len(result)}"

print(vertica_select_frame100_300_order)
result.clear()

Vertica время запроса frame > 100 and < 300 order: 2.342996120452881 количество строк в ответе: 99676


In [40]:
cursor.execute("DROP TABLE test;")

# Итог

In [41]:
print(clickhouse_insert,
      "",
clickhouse_insert3,
      "",
clickhouse_insert3_iterate,
      "",
clickhouse_insert500,
      "",
clickhouse_insert500_iterate,
      "",
clickhouse_select_all,
      "",
clickhouse_select_movie_id,
      "",
clickhouse_select_frame1,
      "",
clickhouse_select_frame100_300,
      "",
clickhouse_select_frame100_300_order,
      "",
vertica_insert,
      "",
vertica_insert3,
      "",
vertica_insert3_iterate,
      "",
vertica_insert500,
      "",
vertica_insert500_iterate,
      "",
vertica_select_all,
      "",
vertica_select_movie_id,
      "",
vertica_select_frame1,
      "",
vertica_select_frame100_300,
      "",
vertica_select_frame100_300_order,
      "",
      sep='\n')

Clickhouse вставка из файла: 40.03273797035217

Clickhouse вставка 3 строк: 0.009743928909301758

Clickhouse cреднее время вставки 3 строк 1000 раз: 0.01030745153799998)

Clickhouse время вставки 500 строк: 0.012792825698852539

Clickhouse cреднее время вставки 500 строк 1000 раз: 0.011356499548000016

Clickhouse время запроса всех строк: 60.1036102771759 kоличество строк в ответе: 10503428

Clickhouse время запроса по конкретному movie_id: 0.08955597877502441 количество строк в ответе: 1001

Clickhouse время запроса frame > 1000: 27.881654024124146 количество строк в ответе: 5253700

Clickhouse время запроса frame > 100 and < 300: 0.6122901439666748 количество строк в ответе: 103581

Clickhouse время запроса frame > 100 and < 300 order: 0.748755931854248 количество строк в ответе: 103581

Vertica вставка из файла: 38.8112678527832

Vertica вставка 3 строк: 0.14068317413330078

Vertica cреднее время вставки 3 строк 1000 раз: 0.043370120616000006)

Vertica время вставки 500 строк: 4.978

### По результатам clickhouse работает быстрее vertica, так же из результатов видно что в clickhouse время вставки 3 строк занимает практически такое же время как и вставка 500 строк поэтому данные стоит вставлять пачками.